*Universidad de Chile*  
*Facultad de Ciencias Físicas y Matemáticas*  
*Departamento de Ingeniería Matemática*


**MA5701 - Optimización no lineal. Semestre de Otoño 2022**  
**Profesor:** Jorge Amaya.    
**Auxiliar:** Aldo Gutierrez.     
**Ayudantes:** Carolina Chiu, Mariano Vazquez.     
**Alumno:** Manuel Torres.

In [1]:
# (Des)comentar la primera vez que se hace correr el codigo
pip install numdifftools

  Using cached numdifftools-0.9.40-py2.py3-none-any.whl (99 kB)
  Using cached algopy-0.5.7-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Librerias
import numpy as np
from numpy import linalg
import scipy as sp
from scipy.optimize import linprog, minimize, LinearConstraint
import numdifftools as nd

C:\Users\Personal\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Personal\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Personal\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Metodo de direcciones admisibles
Se busca resolver problemas de optimización de la forma:
\begin{equation*}
    \begin{array}{cc}
        (P) & \min f(x)  \\
            & Ax \leq b  \\
            & Ex = e.
    \end{array}
\end{equation*}

$(0)$ Sean $\varepsilon>0$, $k=0$, $x_{0}\in\mathbb{R}^{n}$ tal que $Ax_{0}\leq b$, $Ex_{0}=e$.
        
$(1)$ Sea la descomposición
\begin{equation*}
    A=
    \begin{bmatrix}
        A_{1}\\A_{2}
    \end{bmatrix},\quad
    b =
    \begin{bmatrix}
        b_{1}\\b_{2}
    \end{bmatrix},
\end{equation*}
tal que $A_{1}x_{k} = b_{1}$, $A_{2}x_{k} <b_{2}$.
        
$(2)$ Resolver el problema lineal
\begin{equation*}
    (\mathcal{D}_{k})\left\{
    \begin{array}{cl}
        \min        & \nabla f(x_{k})^{T}d  \\
        \text{s.a.} & A_{1}d\leq0           \\
                    &   Ed = 0\\
                    &   -1\leq d_{j} \leq 1,\quad j=1,\dots,n.
    \end{array}
    \right.
\end{equation*}
y sea $d_{k}$ solución de $(\mathcal{D}_{k})$.     
-Si $\|\nabla f(x_{k})^{T}d_{k}\|<\varepsilon$, entonces parar.    
-En caso contrario, ir a $(3)$.


$(3)$ Determinar el paso, resolviendo aproximadamente el problema  de minimización unidimensional
\begin{equation*}
    (L)\left\{
    \begin{array}{cl}
        \min        & f(x_{k}+\lambda d_{k}) \\
        \text{s.a}  & \lambda\in[0,\lambda'_{k}] 
    \end{array}
    \right.
\end{equation*}
mediate el *método de Armijo*. Se usa
\begin{equation*}
    \lambda'_{k} = \min\left\{\frac{(b_{2}-A_{2}x_{k})_{i}}{(A_{2}d_{k})_{i}} / (A_{2}d_{k})_{i} > 0\right\},
\end{equation*}
y se considera $\lambda'_{k}=+\infty$ cuando $(A_{2}d_{k})_{i}\leq0$ para todo $i$.

Sea $\lambda_{k}$ solución del subproblema $(L)$. Hacer:
\begin{equation*}
    x_{k+1} = x_{k} + \lambda_{k}d_{k},\quad k\leftarrow k+1
\end{equation*}
e ir a $(1)$.

# Implementación del método de direcciones admisibles
## Métodos auxiliares
Para los pasos $(1)$, $(2)$ y $(3)$ se implementan los siguientes métodos:
1. *desigualdades_activas_inactivas* realiza la partición sobre $A$ y $b$.
2. *problema_d_k* resuelve el problema $\mathcal{D}_{k}$.
3. *metodo_de_armijo* resuelve el problema $L$ mediante el método de Armijo.

In [122]:
def desigualdades_activas_inactivas(A,b,x):
    """
    -Input: Sistema de inecuaciones Ax <= b.
    -Output: A1,A2,b1,b2 typo np.array.
    -Descripcion: Particiona el sistema de inecuaciones Ax <= b en las desigualdades
    activas y las desigualdades inactivas.
    """
    # Declara los arreglos para guardar la particion
    A1,A2,b1,b2 = [],[],[],[]
    for i in range(len(A)):
        # Igualdades alcanzadas
        if np.isclose(A[i]@x, b[i]):
            A1.append(A[i].tolist())
            b1.append(b[i].tolist())
        # Desigualdades que no alcanzan igualdad
        elif A[i]@x<b[i]:
            A2.append(A[i].tolist())
            b2.append(b[i].tolist())
    # Output en formato de arreglos
    #return = A1,A2,b1,b2 
    # Output en formato de arreglos de numpy (se transforman de array a np.array)
    return np.array(A1), np.array(A2), np.array(b1), np.array(b2)
def problema_d_k(f,xk,A1,E):
    """
    -Input:
    -Output:
    -Descripcion:
    """
    # Calcula el gradiente de f utilizando la libreria numdifftools
    gradiente        = nd.Gradient(f)
    # Funcion objetivo: Se define a partir del vector gradiente
    funcion_objetivo = lambda d: gradiente(xk)@d
    # Cotas para xk
    cotas            = tuple([i for i in zip([-1 for _ in range(len(xk))], 
                                             [1 for _ in range(len(xk))])])
    restricciones    = []
    restricciones_E  = []
    # Si no hay restricciones del estilo Ex = 0
    if E is None:
        for i in range(len(A1)):
            restricciones.append(LinearConstraint(A1,[-np.inf]*A1.shape[0],[0]*A1.shape[0]))
    # Si hay restricciones del estilo Ex = 0
    else:
        for i in range(len(E)):
            restricciones_E.append(0)
        restricciones.append(LinearConstraint(E,restricciones_E,restricciones_E))
    # Metodo optimizador: Se utiliza scipy.optimize.minimize
    resultado = minimize(funcion_objetivo,[1]*len(xk),method='trust-constr',bounds=cotas,constraints=restricciones)
    argmin    = resultado.x # Output
    return argmin
def metodo_de_armijo(f,gradiente,xk,dk,A2,b2,A,b):
    """
    -Input:
    -Output:
    -Descripcion:
    """
    sig = 0.6
    h   = 0.01
    m   = 1
    lambdas = []
    uwu     = []
    for i in range(b2.shape[0]): 
        if np.all(A2@dk <= 0) == True:
            lambdas.append(np.inf)
        elif A2[i]@dk > 0:
            uwu.append((b2[i] - np.array(A2[i])@ xk)/(A2[i]@dk))
    lambdas.append(min(uwu))    
    while h * m <= lambdas[0]:
        if f(xk) + sig*m*h*gradiente(xk) @ dk >= f(xk + m*h*dk):
            m+=1
        else:
            if np.all(A@(xk+h*(m-1)*dk) <= b) == True:
                return h*(m-1)
            else:
                m-=1
    else:
        return lambdas[0]

## Método principal
A continuación se implementa el algoritmo con apoyo de los métodos auxiliares antes implementados. Recordar que las instancias iniciales necesarias son $\varepsilon>0$, $k=0$, $x_{0}\in\mathbb{R}^{n}$ tal que $Ax_{0}\leq b$, $Ex_{0}=e$.

In [130]:
def metodo_direcciones_admisibles(eps,x0,f,A,b,E=None,e=None,max_cantidad_iteraciones=100):
    """
    -Input:
    -Output:
    -Descripcion:
    """
    # Contador de cantidad de iteraciones realizadas / conv = True mientras k no exceda
    # la cantidad maxima de iteraciones
    k, conv = 0, True  
    # Punto inicial (viene propuesto un punto para comenzar junto con el problema)
    xk = x0
    "Iteración inicial"
    # Paso (1)
    A1, A2, b1, b2 = desigualdades_activas_inactivas(A, b, xk)
    # Paso (2)
    dk = problema_d_k(f, xk, A1, E)    
    "Iteraciones hasta cumplir la condicion \|\nabla f(x_{k})^{T}d_{k}\|<\epsilon"
    while np.abs(nd.Gradient(f)(xk) @ dk) > eps:
        # Pasada una cantidad maxima de iteraciones se supondra que el problema
        # es irresoluble con el metodo de direcciones admisibles.
        if k > max_cantidad_iteraciones:
            print('Se agotó el máximo de iteraciones ({})'.format(max_iter))
            conv = False
            break
        # Paso (3)
        tk = metodo_de_armijo(f, nd.Gradient(f), xk,dk, A2, b2, A, b)
        if tk == 0:break        
        xk = xk + tk*dk
        # Aumenta el contador de iteraciones
        k = k+1 
        # Paso (1)
        A1, A2, b1, b2 = desigualdades_activas_inactivas(A, b, xk)
        # Paso (2)
        dk = problema_d_k(f, xk, A1, E)
    "Entregar reporte de los resultados"
    print('Resultados:')
    if conv:
        print('Se alcanzó el óptimo en {} iteraciones'.format(k))
        print('Valor óptimo: {}'.format(f(xk)))
        print('Solución: {}'.format(xk))
    else:
        print('Se excede la cantidad máxima de iteraciones.')
        print('El valor obtenido antes de alcanzar el máximo de iteraciones es: {}'.format(f(xk)))
        print('La solución obtenida antes de alcanzar el máximo de iteraciones es: {}'.format(xk))
    output = xk
    return output

## Problemas test
Se testea el método de direcciones admisibles con los siguientes problemas test:
1. Comenzando del punto $(0,2)$,
\begin{equation*}
    (P_{1})\left\{
    \begin{array}{crcl}
        \min        & 8(x_{1}-6)^{2}+(x_{2}-2)^{4}  &&\\
        \text{s.a.} & -x_{1}+2x_{2}             &\leq& 4\\
                    & 3x_{1}+2x_{2}             &\leq& 12\\
                    & x_{1},x_{2}               &\geq& 0
    \end{array}\right.
\end{equation*}

In [132]:
"Problema 1 - Test"
# Punto inicial
x0 = np.array([0,2])
# Problema P_1
f = lambda x : 8*(x[0] - 6)**2 + (x[1] - 2)**4
A = np.array([[-1, 2], [3, 2], [-1, 0], [0, -1]])
b = np.array([4,12,0,0])
# Metodo de direcciones admisibles sobre P1
sol1 = metodo_direcciones_admisibles(0.001,x0,f,A,b)

C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of usin

Resultados:
Se alcanzó el óptimo en 3 iteraciones
Valor óptimo: 46.9041227202609
Solución: [3.85346355 0.21980435]


C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of usin

2. Comenzando del punto $(2,2,3,2)$,
\begin{equation*}
    (P_{2})\left\{
    \begin{array}{crcl}
        \min        & x_{1}^{4}-2x_{2}^{2}+10x_{1}x_{2}^{2}+x_{4}^{2}  &&\\
        \text{s.a.} & x_{1}+x_{2}-x_{3}             &=& 1\\
                    & x_{1}                         &=& 4\\
                    & x_{1}+x_{4}                   &=& 0\\
                    & x_{1},x_{2},x_{3},x_{4}       &\geq& 0
    \end{array}\right.
\end{equation*}

In [136]:
"Problema 2 - Test"
# Punto inicial
x0 = np.array([2, 2, 3, 2])
# Problema P_2
g = lambda x : x[0]**4 - 2*x[1]**2 + 10*x[0]*x[1]**2 + x[3]**2
A = np.array([[-1,0,0,0], [0,-1,0,0], [0,0,-1,0], [0,0,0,-1]])
E = np.array([[1,1,-1,0], [1,0,0,1], [1,-1,0,0]])
b = np.array([0, 0, 0, 0])
e = np.array([1, 4, 0])
# Metodo de direcciones admisibles sobre P2
sol2 = metodo_direcciones_admisibles(0.1,x0,g,A,b,E,e)

C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of usin

Resultados:
Se alcanzó el óptimo en 5 iteraciones
Valor óptimo: 13.047127462172996
Solución: [0.5358624  0.5358624  0.07172479 3.4641376 ]


C:\Users\Personal\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


# Referencias
- Numdifftools’ documentation: https://numdifftools.readthedocs.io/en/latest/index.html (WebPage), https://github.com/pbrod/numdifftools (GitHub).
- Numerical Optimization Algorithms: Step Size Via the Armijo Rule: https://www.youtube.com/watch?v=Uz3B9fVb4LQ (Tutorial).
